In [1]:
from batid.services.guess_bdg_new import Guesser,GeocodeAddressHandler
from batid.models import Building
import csv
import os

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
def row_to_input(line_number, row) -> dict:
    
    address_infos = [
        row['num_voie'],
        row['ind_rep'],
        row['nom_voie'],
        row['code_postal'],
        row['commune']
    ]
    
    address_infos = [i for i in address_infos if i]
    address = " ".join(address_infos)
    
    return {
        'ext_id': f"{row['num_arrete']}--ligne{line_number}",
        'address': address.lower()
    }
    

In [3]:
os.remove("guess.json")

with open("historique_insa_rnb.csv", "r") as f:
    
    reader = csv.DictReader(f)
    inputs = []
    
    for i, row in enumerate(list(reader)):
        inputs.append(row_to_input(i+2, row))
        
    
    guesser = Guesser()
    guesser.load_inputs(inputs)
    guesser.save_work_file('guess.json')

In [4]:
guesser = Guesser()


guesser.handlers = [GeocodeAddressHandler()]
guesser.guess_work_file('guess.json')

guesser.report()




-- Report --
Number of rows: 277
Number of match: 230 (83.03%)

-- match_reasons : absolute --
precise_address_match    230
Name: match_reason, dtype: int64

-- match_reasons : % --
precise_address_match    83.032491
Name: match_reason, dtype: float64

-- Inputs --
rows with ban_id: 233 (84.12%)


In [5]:
guesser.display_unmatched(30)

    matches match_reason     finished_steps        input_ext_id  \
150      []         None  [geocode_address]  2022-128--ligne152   
64       []         None  [geocode_address]    2022-15--ligne66   
182      []         None  [geocode_address]   2023-25--ligne184   
261      []         None  [geocode_address]  2023-124--ligne263   
199      []         None  [geocode_address]   2023-42--ligne201   
258      []         None  [geocode_address]  2023-117--ligne260   
201      []         None  [geocode_address]   2023-47--ligne203   
127      []         None  [geocode_address]   2022-92--ligne129   
75       []         None  [geocode_address]    2022-26--ligne77   
155      []         None  [geocode_address]  2022-134--ligne157   
67       []         None  [geocode_address]    2022-18--ligne69   
57       []         None  [geocode_address]    2022-07--ligne59   
21       []         None  [geocode_address]    2021-29--ligne23   
18       []         None  [geocode_address]    2021-22--ligne2

In [6]:
bdgs = Building.objects.filter(addresses__id='13054_0530_00007')
print(bdgs.count())

1
